In [1]:
import math
import os
import shutil
from os import listdir, mkdir
from os.path import isfile, join
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import seaborn as sns
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
def getCsiMatrFromCsv(fileName: str, CsiFilesFolder: str) -> pd.DataFrame:
    csvFilePath = join(CsiFilesFolder, fileName)
    return pd.read_csv(csvFilePath)

### read train, test datasets

In [3]:
dataset_path = "dataset1/dataset"
train_dataset = getCsiMatrFromCsv("train.csv", dataset_path)
train_dataset = train_dataset.rename(columns={train_dataset.columns[-1]: "target"})
test_dataset = getCsiMatrFromCsv("test.csv", dataset_path)
test_dataset = test_dataset.rename(columns={test_dataset.columns[-1]: "target"})


In [4]:
train_target = train_dataset.target
test_target = test_dataset.target

le = preprocessing.LabelEncoder()
le.fit(train_target)
train_target = pd.DataFrame(le.transform(train_target))

le = preprocessing.LabelEncoder()
le.fit(test_target)
test_target = pd.DataFrame(le.transform(test_target))

train_features = train_dataset.drop(columns=["target", ])
test_features = test_dataset.drop(columns=["target", ])

In [5]:
dataset = pd.concat([train_features, test_features])
dataset
target = pd.concat([train_target, test_target])
target

,0
0,0
1,0
2,0
3,0
4,0
...,...
43,1
44,1
45,1
46,1


### 1) Random Forest


In [63]:
model = RandomForestClassifier(oob_score=True, random_state=1, n_jobs=-1)
# print(model.get_params())

In [64]:
param_grid = { "criterion" : ["entropy"],
               "min_samples_leaf" : [1, 5, 10],
               "min_samples_split" : [2, 4, 10 ,12, 16],
               "max_depth": [6, 8, 10],
               "max_features": ["auto"],
               "bootstrap": [True],
               "n_estimators": [50, 100, 200, 300]}
gs = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy',
                  cv=3, verbose=10)
gs = gs.fit(train_features, train_target)

Fitting 3 folds for each of 180 candidates, totalling 540 fits
[CV 1/3; 1/180] START bootstrap=True, criterion=entropy, max_depth=6, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 1/3; 1/180] END bootstrap=True, criterion=entropy, max_depth=6, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.625 total time=   1.2s
[CV 2/3; 1/180] START bootstrap=True, criterion=entropy, max_depth=6, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 2/3; 1/180] END bootstrap=True, criterion=entropy, max_depth=6, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.538 total time=   1.3s
[CV 3/3; 1/180] START bootstrap=True, criterion=entropy, max_depth=6, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50


...

d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 2/3; 180/180] END bootstrap=True, criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=16, n_estimators=300;, score=0.513 total time=   3.6s
[CV 3/3; 180/180] START bootstrap=True, criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=16, n_estimators=300


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 3/3; 180/180] END bootstrap=True, criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=16, n_estimators=300;, score=0.462 total time=   3.5s


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [65]:
print(gs.best_score_)
print(gs.best_params_)
# print(gs.cv_results_)
best_model_1 = gs.best_estimator_

0.6021367521367522
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 6, 'max_features': 'auto', 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 100}


In [66]:
# print(best_model_1.score(test_features, test_target))
print("score=", best_model_1.score(test_features, test_target))
scs = cross_val_score(best_model_1, dataset, target, cv=5, scoring="accuracy")
print("cross_val_score=", scs)
print("mean=", scs.mean())

score= 0.4791666666666667


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
d:\googledrive\wirelessmo

cross_val_score= [0.47058824 0.51515152 0.54545455 0.51515152 0.60606061]
mean= 0.5304812834224599


### 2) Naive Bayes

In [67]:
gnb_model = GaussianNB()
# print(model.get_params())

In [68]:
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}
gs = GridSearchCV(estimator=gnb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=10, n_jobs=-1)
gs = gs.fit(train_features, train_target)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\naive_bayes.py", line 245, in fit
    return self._partial_fit(
  File "d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packag

In [69]:
print(gs.best_score_)
print(gs.best_params_)
# print(gs.cv_results_)
best_model_2 = gs.best_estimator_

0.7032051282051283
{'var_smoothing': 0.001873817422860383}


In [70]:
print("score=", best_model_2.score(test_features, test_target))
scs = cross_val_score(best_model_2, dataset, target, cv=7, scoring="accuracy")
print("cross_val_score=", scs)
print("mean=", scs.mean())

score= 0.5833333333333334


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\utils\validati

cross_val_score= [0.70833333 0.70833333 0.66666667 0.83333333 0.58333333 0.52173913
 0.60869565]
mean= 0.6614906832298137


### 3) K-nearest neighbor

In [6]:
neigh_model = KNeighborsClassifier()
# print(model.get_params())

In [7]:
param_grid = {
    'n_neighbors': (1, 5, 6, 9, 10, 20),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')
               }
gs = GridSearchCV(estimator=neigh_model, param_grid=param_grid,
                    cv=3, scoring="accuracy", verbose=10, n_jobs=-1)
gs.fit(X=train_features, y=train_target)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'leaf_size': (20, 40, 1),
                         'metric': ('minkowski', 'chebyshev'),
                         'n_neighbors': (1, 5, 6, 9, 10, 20), 'p': (1, 2),
                         'weights': ('uniform', 'distance')},
             scoring='accuracy', verbose=10)

In [8]:
print(gs.best_params_)
print(gs.best_score_)
best_model_3 = gs.best_estimator_

{'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
0.5589743589743589


In [9]:
print("score=", best_model_3.score(test_features, test_target))
scs = cross_val_score(best_model_3, dataset, target, cv=7, scoring="accuracy")
print("cross_val_score=", scs)
print("mean=", scs.mean())

score= 0.4166666666666667


d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\googledrive\wirelessmotionintelligence\2022-knu-wmi\venv\lib\site-packages\sklearn\neighbors\_cla

cross_val_score= [0.54166667 0.41666667 0.54166667 0.45833333 0.54166667 0.34782609
 0.39130435]
mean= 0.46273291925465837


In [ ]:
train_target